In [1]:
from src.base import Block, Boxtype
import src.vcs_function as eval
import src.box_plotter as bxplot

#test Blocks
b1 = Block(boxtype=Boxtype(1,2,4,True,True,True,1), rot='wlh')
b2 = Block(boxtype=Boxtype(1,2,3,True,True,True,1), rot='lwh')
b3 = Block(boxtype=Boxtype(1,1,2,True,True,True,1), rot='hwl')

#test join
print("b1:",b1); print("b2:",b2); print("b3:",b3)
t=b1.join(b2, 'y'); print(t,"b1+b2:",b1)
t=b1.join(b3, 'z'); print(t,"b1+b3:",b1)
t=b1.join(b1, 'x'); print(t,"b1+b1:",b1)

b1: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x000002406424FEB0>: 1} ratio:1.0
b2: Block: l: 2 w: 3 h: True weight: 1 volume: 6 occupied_volume: 6 items: {<src.base.Boxtype object at 0x000002406424FF10>: 1} ratio:1.0
b3: Block: l: True w: 2 h: 1 weight: 1 volume: 2 occupied_volume: 2 items: {<src.base.Boxtype object at 0x000002406424FC40>: 1} ratio:1.0
False b1+b2: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x000002406424FEB0>: 1} ratio:1.0
False b1+b3: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x000002406424FEB0>: 1} ratio:1.0
True b1+b1: Block: l: 8 w: 2 h: True weight: 2 volume: 16 occupied_volume: 16 items: {<src.base.Boxtype object at 0x000002406424FEB0>: 2} ratio:1.0


## Plot container

In [2]:
def load_instance(filename = "instancia.txt", type="BF", id_instance=0):
    with open(filename, "r") as file:
        #discard first line
        file.readline()
      
        for j in range(100):
            file.readline()
            L, W, H = [int(x) for x in file.readline().split()]

            #read second line
            n = int(file.readline())
            #read n lines
            items = Itemdict()
            for i in range(n):
                id, l, rotx, w, roty, h, rotz, n = [int(x) for x in file.readline().split()]    
                item = Boxtype(id, l, w, h, rotx, roty, rotz, 1)
                items[item]=n
            
            if j == id_instance: return  items, L, W, H


In [3]:
def adyacent_blocks(blocks,L,W,H):
    last = blocks[-1]
    adyacent = list()
    box_dims = []
    
    for block in blocks[:-1]:
        
        if block.xmax == last.xmin and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.ymin < last.ymax and block.ymax > last.ymin )):
            z_diff_max = max([block.zmin,last.zmin])
            z_diff_min = min([block.zmax,last.zmax])
            y_diff_max = max([block.ymin,last.ymin])
            y_diff_min = min([block.ymax,last.ymax])
            adyacent.append(block)
            block.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            block.covered_surface_face['X1']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface_face['X2']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)

        elif block.xmin == last.xmax and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.ymin < last.ymax and block.ymax > last.ymin )):
            z_diff_max = max([block.zmin,last.zmin])
            z_diff_min = min([block.zmax,last.zmax])
            y_diff_max = max([block.ymin,last.ymin])
            y_diff_min = min([block.ymax,last.ymax])
            adyacent.append(block)
            block.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            block.covered_surface_face['X2']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface_face['X1']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)

        elif block.ymax == last.ymin and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):
            z_diff_max = max([block.zmin,last.zmin])
            z_diff_min = min([block.zmax,last.zmax])
            x_diff_max = max([block.xmin,last.xmin])
            x_diff_min = min([block.xmax,last.xmax])
            adyacent.append(block)
            block.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            block.covered_surface_face['Y1']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface_face['Y2']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)

        elif block.ymin == last.ymax and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):
            z_diff_max = max([block.zmin,last.zmin])
            z_diff_min = min([block.zmax,last.zmax])
            x_diff_max = max([block.xmin,last.xmin])
            x_diff_min = min([block.xmax,last.xmax])
            adyacent.append(block)
            block.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            block.covered_surface_face['Y2']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface_face['Y1']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)

        elif block.zmax == last.zmin and ((block.ymin < last.ymax and block.ymax > last.ymin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):
            y_diff_max = max([block.ymin,last.ymin])
            y_diff_min = min([block.ymax,last.ymax])
            x_diff_max = max([block.xmin,last.xmin])
            x_diff_min = min([block.xmax,last.xmax])
            adyacent.append(block)
            block.covered_surface+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            # block.covered_surface_face['Z1']+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            # last.covered_surface_face['Z2']+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)

        elif block.zmin == last.zmax and ((block.ymin < last.ymax and block.ymax > last.ymin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):
            y_diff_max = max([block.ymin,last.ymin])
            y_diff_min = min([block.ymax,last.ymax])
            x_diff_max = max([block.xmin,last.xmin])
            x_diff_min = min([block.xmax,last.xmax])
            adyacent.append(block)
            block.covered_surface+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            # block.covered_surface_face['Z2']+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            # last.covered_surface_face['Z1']+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            
    # if(len(adyacent)!=0):
        # print(last.covered_surface_face)
        # box_dims.append([last.xmin,last.ymin,last.zmin,last.xmax,last.ymax,last.zmax])
        # for aabb in adyacent:
        #     box_dims.append([aabb.xmin,aabb.ymin,aabb.zmin,aabb.xmax,aabb.ymax,aabb.zmax])
        # bxplot.plot_container([L,W,H], box_dims)
    pass

# The main code

In [4]:

from src.base import Itemdict, BlockList, Aabb, Space

Space.filling = "bottom-up" # spaces are filled from bottom to top
Space.vertical_stability = True # boxes must be completly supported

items, L, W, H = load_instance(filename = "src/BR4.txt", type="BF", id_instance=1)

cont = Block(l=L,w=W,h=H)

# print("generating blocks...")
blocks = BlockList(items, type="general_blocks", cont=cont, min_fr=1.0, max_bl=1000) 

# for x in blocks:
#    print(x.items)

# greedy for single CLP
while True:
    space = cont.free_space.closest_space() #cuboide mas cercano a un vertice inferior del contenedor

    if space is not None:
      block = eval.eval_function(blocks,space,p=0.05,stored_blocks=cont.aabbs,items=items)
      # print("adding block...", block)
      
      cont.add_block(block, space)
      adyacent_blocks(cont.aabbs,L,W,H)
      items -= block.items
      blocks.remove_unconstructable(items)
      cont.free_space.filter(items)
    else:
        break

print(cont)


{<src.base.Boxtype object at 0x000002407AA100A0>: 1}
{<src.base.Boxtype object at 0x000002407AA100A0>: 1}
{<src.base.Boxtype object at 0x000002407AA100A0>: 1}
{<src.base.Boxtype object at 0x000002407AA100A0>: 1}
{<src.base.Boxtype object at 0x000002407AA13D90>: 1}
{<src.base.Boxtype object at 0x000002407AA13D90>: 1}
{<src.base.Boxtype object at 0x000002407AA13D90>: 1}
{<src.base.Boxtype object at 0x000002407AA13D90>: 1}
{<src.base.Boxtype object at 0x000002407AA13D90>: 1}
{<src.base.Boxtype object at 0x000002407AA13D90>: 1}
{<src.base.Boxtype object at 0x000002407AA12380>: 1}
{<src.base.Boxtype object at 0x000002407AA12380>: 1}
{<src.base.Boxtype object at 0x000002407AA12380>: 1}
{<src.base.Boxtype object at 0x000002407AA12380>: 1}
{<src.base.Boxtype object at 0x000002407AA12380>: 1}
{<src.base.Boxtype object at 0x000002407AA12380>: 1}
{<src.base.Boxtype object at 0x000002407AA12350>: 1}
{<src.base.Boxtype object at 0x000002407AA12350>: 1}
{<src.base.Boxtype object at 0x000002407AA1235

# Plot the solution

In [5]:
from src.restrictions import center_of_gravity

box_dims = []
for aabb in cont.aabbs:
  box_dims.append([aabb.xmin,aabb.ymin,aabb.zmin,aabb.xmax,aabb.ymax,aabb.zmax])

bxplot.plot_container([L,W,H], box_dims)
print("distance center of gravity: ",center_of_gravity(cont))



distance center of gravity:  4.5389210760272505
